In [1]:
import pandas as pd
import numpy as np

In [95]:
data = pd.read_csv('C:/Users/shree/Downloads/samplecar.csv')
data.head(10)

,Type,Price,Buy
0,SUV,H,No
1,HB,H,No
2,HB,H,Yes
3,HB,L,No
4,SEDAN,H,Yes
5,SEDAN,L,Yes
6,SUV,L,No
7,SEDAN,H,Yes
8,HB,L,Yes
9,SUV,H,Yes


In [157]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]
y.value_counts().index

Index(['Yes', 'No'], dtype='object')

In [158]:
X

,Type,Price
0,SUV,H
1,HB,H
2,HB,H
3,HB,L
4,SEDAN,H
5,SEDAN,L
6,SUV,L
7,SEDAN,H
8,HB,L
9,SUV,H


In [3]:
data.shape

(10, 3)

This part contains the implementation of entropy and information gain from completely manual approach. I did the manual approach to explain the flow to my students. I have implemented using the function below.

In [4]:
y = data[data['Buy']=='Yes'].shape[0]
n = data[data['Buy']=='No'].shape[0]
total_sample=data.shape[0]
print(y,n)
print(total_sample)

6 4
10


In [5]:
p_y = y/total_sample
p_y

0.6

In [6]:
p_n = n/total_sample
p_n

0.4

In [7]:
p_list=[p_y,p_n]

In [8]:
def prob(i,n):
    return i/n

In [9]:
def entropy(p_l):
    entropy_value = 0
    
    for i in p_l:
        if i == 0:
            continue
            
        entropy_value += -1*(i*np.log2(i))
        
    return entropy_value

In [10]:
parent_entropy = entropy(p_list)
print(parent_entropy)

0.9709505944546686


# Entropy, weighted_entropy and information gain for Type feature

In [11]:
data.Type.value_counts()

HB       4
SUV      3
SEDAN    3
Name: Type, dtype: int64

In [12]:
data.Type.value_counts()[1]

3

In [13]:
len(data)

10

In [14]:
#for HB
hb_no = data[(data['Type']=='HB')&(data['Buy']=='No')].shape[0]
hb_yes = data[(data['Type']=='HB') & (data['Buy']=='Yes')].shape[0]
print(hb_no, hb_yes)

#probability and entropy
p_hb_y = hb_yes/data.Type.value_counts()[0]
p_hb_n = hb_no/data.Type.value_counts()[0]
hb_entropy = entropy([p_hb_y,p_hb_n])
print('entropy of HB:',hb_entropy)

2 2
entropy of HB: 1.0


In [15]:
#for SVU

suv_no=data[(data['Type']=='SUV') & (data['Buy']=='No')].shape[0]
suv_yes = data[(data['Type']=='SUV')&(data['Buy']=="Yes")].shape[0]
print(suv_no, suv_yes)

#probability and entropy
p_suv_y = suv_yes/data.Type.value_counts()[1]
p_suv_n = suv_no/data.Type.value_counts()[1]
suv_entropy = entropy([p_suv_y,p_suv_n])
print('entropy of SUV:',suv_entropy)

2 1
entropy of SUV: 0.9182958340544896


In [16]:
#for sedan
sedan_no = data[(data['Type']=='SEDAN')&(data['Buy']=='N0')].shape[0]
sedan_yes = data[(data['Type']=='SEDAN')& (data['Buy']=='Yes')].shape[0]
print(sedan_no, sedan_yes)

#probability and entropy
p_s_y = sedan_yes/data.Type.value_counts()[2]
p_s_n = sedan_no/data.Type.value_counts()[2]
s_entropy = entropy([p_s_y,p_s_n])
print('entropy of SEDAN:',s_entropy)

0 3
entropy of SEDAN: 0.0


Weighted entropy of Type feature.

In [17]:
#weighted entropy of Type feature
w_hb=(data.Type.value_counts()[0]/data.shape[0])*hb_entropy
w_suv=(data.Type.value_counts()[1]/data.shape[0])*suv_entropy
w_sedan=(data.Type.value_counts()[2]/data.shape[0])*s_entropy
w_entropy_type = w_hb + w_suv + w_sedan
w_entropy_type

0.6754887502163469

Information Gain of Type feature.

In [18]:
#information gain
information_gain_type = parent_entropy - w_entropy_type
information_gain_type

0.29546184423832167

# Below, I have tried to implement everything by using two functions.

Yes, I know its not the optimized code. I will try to make it optimize in near future.

In [141]:
def entropy(p_l):
    entropy_value = 0
    
    for i in p_l:
        if i == 0:
            continue
            
        entropy_value += -1*(i*np.log2(i))
        
    return(entropy_value)

In [199]:
def weighted_entropy_features(X,y):
    g = [(len(y[y==i])/len(y)) for i in y.value_counts().index] 
    #calculating the probability of target column to calculate parent entropy.

    for h in X.columns:
        print(f'Calculating for feature: {h}')
        print()
        weighted_entropy=0
        for i in X[h].value_counts().index:
            b = len(X[X[h]==i])
            print(f'Total number of {i} in {h}:',b)
            l = []
            for j in y.value_counts().index:              
                a=X[(X[h]==i)&(y==j)].shape[0]
                print(f'Total number of {j} in {i}:',a)
                l.append(round(a/b,2)) #calculating the probability and taking the .2f value    
                
            e = entropy(l) #calculating the entropy
            print(f'The entropy of {i} =',e)
            print()
            
            weighted_entropy += (b/len(X))* e
        print(f'The weighted entropy of {h} = ',weighted_entropy)
        
        p_e = entropy(g) #calculating the parent entropy to calculate the information gain.
        
        information_gain= p_e-weighted_entropy
        print(f'The information gain of {h}= {information_gain}')
        print('***************') 
weighted_entropy_features(X,y)

Calculating for feature: Type

Total number of HB in Type: 4
Total number of Yes in HB: 2
Total number of No in HB: 2
The entropy of HB = 1.0

Total number of SUV in Type: 3
Total number of Yes in SUV: 1
Total number of No in SUV: 2
The entropy of SUV = 0.9149263727797274

Total number of SEDAN in Type: 3
Total number of Yes in SEDAN: 3
Total number of No in SEDAN: 0
The entropy of SEDAN = 0.0

The weighted entropy of Type =  0.6744779118339183
The information gain of Type= 0.2964726826207503
***************
Calculating for feature: Price

Total number of H in Price: 6
Total number of Yes in H: 4
Total number of No in H: 2
The entropy of H = 0.9149263727797274

Total number of L in Price: 4
Total number of Yes in L: 2
Total number of No in L: 2
The entropy of L = 1.0

The weighted entropy of Price =  0.9489558236678365
The information gain of Price= 0.021994770786832096
***************
